# RSA Encryption
***
## Sending information to your bank over a public channel using encruption

### Steps
1. The bank determines two secret prime numbers, p and q, these mulitply to be N, so $N = p \times q$. N is a public key, p and q are bank secrets
2. Find $\phi(N) = (p-1) \times (q-1)$
3. Find d and e, d is a bank secret key and e is a public key, so:
    - $K = e \times d$
    - $K mod \phi(N) = 1$ 
    - $K = 1 mod \phi(N) = e \times d$
5. K factors into d * e
6. Bank distributes N and e as public keys, Bank keeps p, q, d, $\phi(N)$ as secret keys
7. Convert string into a list of numbers, each number is M
8. Users encrypt each number, $Enc = M^e mod N$ and places each number into a list, the list is sent to the bank
9. Bank decrypts each integer in the list message user $M = Enc^d mod N$, and then rebuilds the string 
10. Bank decodes the numbers back into characters

## Functions

#### Factorising a number
The below script factorises a number and returns the first result it finds, it starts from the 2 (because 1 will go into any number) and do an itterative factor tree, up until the value of K. 

In [2]:
from random import randint

def factor_script(k):
    fact_list = []
    found = False

    for x in range(2, k):
        if k % x == 0 and found == False:
            fact_list.append((int(k/x),x))
            found = True
            
        if found == True:
            break
    
    return fact_list

#### Finding K and then d and e

Formula 1: $K mod \phi(N) = 1$

Formula 2: $K = 1 mod \phi(N) = e \times d$

Using these formulae, K is found first and it the script checks if K can be factorised, if it can it is then factorised into d and e, if not it discards that value of K and intterates until it finds the next candidate.

In [3]:
def find_de_as_k(phi_pq, prime1, prime2, factor_script_func):
    # Generate candidate list for k = d*e
    de_list = []

    # Set k as the maximum of prime1 or prime2
    k = max(prime1, prime2)

    # Loop through and increase k options by 1 every time, and check criteria for eligability
    found_de = False
    while found_de == False:

        # Factorises K into d*e and returns it if it exists, returns an empty list if it can't be factorised
        de = factor_script_func(k)

        # Check that k * modphi(N) = 1 and that k has factors
        if (k) % phi_pq == 1 and de != []:

            # Select the first candidate for K and end the loop
            de_list.append(de)
            found_de = True

        k += 1

    # Return k as a list make up of (d, e)
    return de_list

#### Encoding the String

This script uses the attached dictionary to encode a string into specific integers, this allows the numbers to be then encrypted and decrypted.

In [4]:
# Encoding dictionary
str_dict = {'a': "1", 'b': "2", 'c': "3", 'd': "4", 'e': "5", 'f': "6", 'g': "7", 'h': "8", 'i': "9", 
            'j': "10", 'k': "11", 'l': "12", 'm': "13", 'n': "14", 'o': "15", 'p': "16", 'q': "17", 
            'r': "18", 's': "19", 't': "20", 'u': "21", 'v': "22", 'w': "23", 'x': "24", 'y': "25", 
            'z': "26", ' ': "27"}


# Encodes the string (M) and encrypts it with Enc = M^e mod N, where M is the message as an integer
def encrypt_string(string_in, bank_code1,bank_code2):

    string_as_num = [(int(str_dict[item])**bank_code1)%bank_code2 for item in string_in]

    return string_as_num


# Decrypt a list of encrypted characters and return the message M, M = Enc^d mod N
def decrypt_string(encrypted_str, bank_secret, bank_code2):

    # Decrypts the message into the encoded list
    decrypted_list = [(item**bank_secret)%bank_code2 for item in encrypted_str]    
    
    # Matches the key with the letter and appends it into a string
    string_out = ""
    for char in decrypted_list:
        string_out += "".join([k for k,v in str_dict.items() if v == str(char)][0])


    return string_out

#### Generate two prime numbers

The below script using the fact that the only number that devides a prime number is itself and 1 with no remainders to itterate through a range and return a list of prime numbers. The script then randomly selects two numbers ensuring the seccond number is bigger than the first one to select two prime numbers and return them.

In [5]:
# Generate two prime numbers
def gen_prime(start, stop):
    mod_list = []

    # Generate list of prime numbers
    for num in range(start, stop):
        if num > 1:
            for i in range(2, num):
                if (num % i) == 0:
                    break
            else:
                mod_list.append(num)

    # Randomly pick a number to pick a number from the list
    x = randint(1,len(mod_list)-1)
    y = randint(x,len(mod_list)-1)

    return mod_list[x], mod_list[y]

### Main Script

This script runs the encryption algorithm and is a type of mock interface that shows it all works in motion.

In [6]:
def main():
    # Banks secret keys - Two primes that make up N
    prime1, prime2 = gen_prime(3,200)
    print(f"Two secret primes that make up N = p*q = {prime1} * {prime2} = {prime1 * prime2} where N is a public key and p and q are bank secrets.\n")

    # Find phi(N) = phi(pq)
    phi_pq = (prime1-1) * (prime2-1)
    print(f"Phi(N) = (p - 1) * (q - 1)= {phi_pq}\n")

    # Find K
    # K = 1mod(fi(N))- K has to be able to be factored
    de = find_de_as_k(phi_pq, prime1, prime2,factor_script)[0]
    d, e = de[0]
    print(f"K = 1mod(Phi(N)) = d*e = {d} * {e}.\n")
    print(f"So, d = {d} and e = {e}. Then, 'd' becomes the banks secret key and 'e' is the other public key.\n")

    # Banks other secret key is d
    bank_secret = d

    # Bank public keys which are e and N
    bank_code1 = e
    bank_code2 = prime1 * prime2
    
    print(f"So, we have built the banks two public keys, which are {bank_code1} and {bank_code2}, these are used by senders to encrypt data.\n")
    print(f"As a result of this one way modular maths, it is essentially impossible to reverse the encryption without the private keys.\n")

    # Raw message
    message_raw = "this is a test"

    # Convert message into list and then encrypts each letter.
    encr_list = encrypt_string(message_raw, bank_code1, bank_code2)
    print(f"Encrypted message: {encr_list}\n")

    # Decrypt message
    print(f"Decrypted message: {decrypt_string(encr_list, bank_secret, bank_code2)}")

In [7]:
main()

Two secret primes that make up N = p*q = 137 * 149 = 20413 where N is a public key and p and q are bank secrets.

Phi(N) = (p - 1) * (q - 1)= 20128

K = 1mod(Phi(N)) = d*e = 13419 * 3.

So, d = 13419 and e = 3. Then, 'd' becomes the banks secret key and 'e' is the other public key.

So, we have built the banks two public keys, which are 3 and 20413, these are used by senders to encrypt data.

As a result of this one way modular maths, it is essentially impossible to reverse the encryption without the private keys.

Encrypted message: [8000, 512, 729, 6859, 19683, 729, 6859, 19683, 1, 19683, 8000, 125, 6859, 8000]

Decrypted message: this is a test
